
### Question 1: [Index] S&P 500 Stocks Added to the Index

**Which year had the highest number of additions?**

Using the list of S&P 500 companies from Wikipedia's [S&P 500 companies page](https://en.wikipedia.org/wiki/List_of_S%26P_500_companies), download the data including the year each company was added to the index.

Hint: you can use [pandas.read_html](https://pandas.pydata.org/docs/reference/api/pandas.read_html.html) to scrape the data into a DataFrame.

Steps:
1. Create a DataFrame with company tickers, names, and the year they were added.
2. Extract the year from the addition date and calculate the number of stocks added each year.
3. Which year had the highest number of additions (1957 doesn't count, as it was the year when the S&P 500 index was founded)? Write down this year as your answer (the most recent one, if you have several records).

*Context*: 
> "Following the announcement, all four new entrants saw their stock prices rise in extended trading on Friday" - recent examples of S&P 500 additions include DASH, WSM, EXE, TKO in 2025 ([Nasdaq article](https://www.nasdaq.com/articles/sp-500-reshuffle-dash-tko-expe-wsm-join-worth-buying)).

*Additional*: How many current S&P 500 stocks have been in the index for more than 20 years? When stocks are added to the S&P 500, they usually experience a price bump as investors and index funds buy shares following the announcement.

---

In [ ]:


import pandas as pd

In [ ]:
s_p_500 = pd.read_html("https://en.wikipedia.org/wiki/List_of_S%26P_500_companies") 
sp500_table = s_p_500[0]
sp500_table_interest = sp500_table[["Symbol", "Security", "GICS Sector","Date added"]]
symbols_and_dates = sp500_table[['Symbol','Date added']].copy()
#get the number of stocks added each year
symbols_and_dates['Date added'] = pd.to_datetime(symbols_and_dates['Date added'])
symbols_and_dates['Year'] = symbols_and_dates['Date added'].dt.year
added_per_year = symbols_and_dates.groupby('Year').size().reset_index(name='Count')
added_per_year = added_per_year.sort_values(by='Count', ascending=False)


In [ ]:

added_per_year

In [ ]:
#Additional: How many current S&P 500 stocks have been in the index for more than 20 years? 
# When stocks are added to the S&P 500,
# they usually experience a price bump as investors and index funds buy shares following the announcement.
current_date = pd.to_datetime('today')
long_term_stocks = symbols_and_dates[symbols_and_dates['Date added'] <= current_date - pd.Timedelta(days=365*20)]
long_term_count = long_term_stocks['Symbol'].nunique()
print(long_term_count)
long_term_stocks[['Symbol', 'Date added']].head(10)  # Displaying the first 10 long-term stocks


### Question 2. [Macro] Indexes YTD (as of 1 May 2025)

**How many indexes (out of 10) have better year-to-date returns than the US (S&P 500) as of May 1, 2025?**

Using Yahoo Finance World Indices data, compare the year-to-date (YTD) performance (1 January-1 May 2025) of major stock market indexes for the following countries:
* United States - S&P 500 (^GSPC)
* China - Shanghai Composite (000001.SS)
* Hong Kong - HANG SENG INDEX (^HSI)	
* Australia - S&P/ASX 200 (^AXJO)
* India - Nifty 50 (^NSEI)
* Canada - S&P/TSX Composite (^GSPTSE)
* Germany - DAX (^GDAXI)
* United Kingdom - FTSE 100 (^FTSE)
* Japan - Nikkei 225 (^N225)
* Mexico - IPC Mexico (^MXX)
* Brazil - Ibovespa (^BVSP)

*Hint*: use start_date='2025-01-01' and end_date='2025-05-01' when downloading daily data in yfinance

Context: 
> [Global Valuations: Who's Cheap, Who's Not?](https://simplywall.st/article/beyond-the-us-global-markets-after-yet-another-tariff-update) article suggests "Other regions may be growing faster than the US and you need to diversify."

Reference: Yahoo Finance World Indices - https://finance.yahoo.com/world-indices/

*Additional*: How many of these indexes have better returns than the S&P 500 over 3, 5, and 10 year periods? Do you see the same trend?
Note: For simplicity, ignore currency conversion effects.)


In [ ]:
import yfinance as yf
major_stock_list = ['^GSPC', '000001.SS','^HSI', '^AXJO', '^NSEI', '^GSPTSE', '^GDAXI', '^FTSE', '^N225', '^MXX','^BVSP']
start_date = '2025-01-01'
end_date = '2025-05-01'
difference = pd.to_datetime(end_date) - pd.to_datetime(start_date)
data = pd.DataFrame()
for stock in major_stock_list:
    try:
        ticker = yf.Ticker(stock)
        df = ticker.history(period='1d', start=start_date, end=end_date)
        df['Symbol'] = stock
        df_filtered = df[df['Symbol'] == stock]
        df_filtered['ytd_change'] = (df_filtered['Close'].iloc[-1]/df_filtered['Close'] - 1)
        df_filtered['ytd_change'] = df_filtered['ytd_change'].fillna(0)  # Fill NaN values with 0
        df.loc[df_filtered.index, 'ytd_change'] = df_filtered['ytd_change']
        df = df.reset_index()
        data = pd.concat([data, df], ignore_index=True, axis=0)
    except Exception as e:
        print(f"Error downloading {stock}: {e}")


In [ ]:
# 1. Group data by Symbol to work with each index separately
# 2. Get first and last trading day for each index
# 3. Calculate proper YTD returns

# First, let's create a function to calculate YTD returns
def calculate_ytd_returns(df_group):
    # Sort by date to ensure first is earliest and last is latest
    df_sorted = df_group.sort_values('Date')
    # Get first and last close price
    first_price = df_sorted['Close'].iloc[0]
    last_price = df_sorted['Close'].iloc[-1]
    # Calculate return
    ytd_return = (last_price / first_price) - 1
    return pd.Series({'ytd_return': ytd_return, })

# Apply the function to each group of Symbol
ytd_returns = data.groupby('Symbol').apply(calculate_ytd_returns).reset_index()

# Identify the S&P 500 return
sp500_return = ytd_returns[ytd_returns['Symbol'] == '^GSPC']['ytd_return'].values[0]

# Compare other indices to S&P 500
better_than_sp500 = ytd_returns[ytd_returns['ytd_return'] > sp500_return]

# Count how many are better (excluding S&P 500 itself)
num_better = len(better_than_sp500[better_than_sp500['Symbol'] != '^GSPC'])

print(f"Number of indices with better YTD returns than S&P 500: {num_better}")
print("\nYTD Returns for all indices (sorted):")
print(ytd_returns.sort_values('ytd_return', ascending=False))

In [ ]:
ytd_returns.sort_values('ytd_return', ascending=False)

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

plt.figure(figsize=(12, 6))
sns.lineplot(data=data, x='Date', y='ytd_change', hue='Symbol')
plt.title('YTD Change of Major Stock Indices')
plt.xlabel('Date')
plt.ylabel('YTD Change')
plt.legend(title='Symbol', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.tight_layout()
plt.show()


### Question 3. [Index] S&P 500 Market Corrections Analysis


**Calculate the median duration (in days) of significant market corrections in the S&P 500 index.**

For this task, define a correction as an event when a stock index goes down by **more than 5%** from the closest all-time high maximum.

Steps:
1. Download S&P 500 historical data (1950-present) using yfinance
2. Identify all-time high points (where price exceeds all previous prices)
3. For each pair of consecutive all-time highs, find the minimum price in between
4. Calculate drawdown percentages: (high - low) / high × 100
5. Filter for corrections with at least 5% drawdown
6. Calculate the duration in days for each correction period
7. Determine the 25th, 50th (median), and 75th percentiles for correction durations

*Context:* 
> * Investors often wonder about the typical length of market corrections when deciding "when to buy the dip" ([Reddit discussion](https://www.reddit.com/r/investing/comments/1jrqnte/when_are_you_buying_the_dip/?rdt=64135)).

> * [A Wealth of Common Sense - How Often Should You Expect a Stock Market Correction?](https://awealthofcommonsense.com/2022/01/how-often-should-you-expect-a-stock-market-correction/)

*Hint (use this data to compare with your results)*: Here is the list of top 10 largest corrections by drawdown:
* 2007-10-09 to 2009-03-09: 56.8% drawdown over 517 days
* 2000-03-24 to 2002-10-09: 49.1% drawdown over 929 days
* 1973-01-11 to 1974-10-03: 48.2% drawdown over 630 days
* 1968-11-29 to 1970-05-26: 36.1% drawdown over 543 days
* 2020-02-19 to 2020-03-23: 33.9% drawdown over 33 days
* 1987-08-25 to 1987-12-04: 33.5% drawdown over 101 days
* 1961-12-12 to 1962-06-26: 28.0% drawdown over 196 days
* 1980-11-28 to 1982-08-12: 27.1% drawdown over 622 days
* 2022-01-03 to 2022-10-12: 25.4% drawdown over 282 days
* 1966-02-09 to 1966-10-07: 22.2% drawdown over 240 days

In [ ]:
# Get historical data
sp500 = yf.download('^GSPC', start='1950-01-01')['Close']

# Identify correction periods
all_time_highs = sp500.cummax()
sp500['Is_ATH'] = (sp500 == all_time_highs)
ath_dates = sp500.index[sp500['Is_ATH']].tolist()


In [ ]:

corrections = []

for i in range(len(ath_dates)-1):
    start = ath_dates[i]
    end = ath_dates[i+1]
    high = all_time_highs.loc[ath_dates[i]].iloc[0]
    low = sp500.loc[start:end].min().iloc[0]
    
    # Calculate drawdown
    drawdown = (high - low)/high * 100
    if isinstance(drawdown, pd.Series):
        drawdown = drawdown.iloc[0]  # Handle series conversion
    
    if drawdown >= 5:
        min_date = sp500.loc[start:end].idxmin().iloc[0]
        duration = (min_date - start).days
        corrections.append({
            'Start Date': start.strftime('%Y-%m-%d'),
            'End Date': min_date.strftime('%Y-%m-%d'),
            'Duration (Days)': duration,
            'Drawdown %': round(drawdown, 1)
        })

# Create formatted dataframe
corrections_df = pd.DataFrame(corrections)
print(corrections_df.head(10))
corrections_df.sort_values(by='Drawdown %', ascending=False)


In [ ]:
print(corrections_df['Duration (Days)'].describe(percentiles=[.25, .5, .75]))


In [ ]:
corrections_df['Duration (Days)'].plot(kind='hist', bins=10, edgecolor='black')
plt.title('Distribution of Correction Durations')


### Question 4.  [Stocks] Earnings Surprise Analysis for Amazon (AMZN)


**Calculate the median 2-day percentage change in stock prices following positive earnings surprises days.**

Steps:
1. Load earnings data from CSV ([ha1_Amazon.csv](ha1_Amazon.csv)) containing earnings dates, EPS estimates, and actual EPS
2. Download complete historical price data using yfinance
3. Calculate 2-day percentage changes for all historical dates: for each sequence of 3 consecutive trading days (Day 1, Day 2, Day 3), compute the return as Close_Day3 / Close_Day1 - 1. (Assume Day 2 may correspond to the earnings announcement.)
4. Identify positive earnings surprises (where "actual EPS > estimated EPS" OR "Surprise (%)>0")
5. Calculate 2-day percentage changes following positive earnings surprises
6. Compare the median 2-day percentage change for positive surprises vs. all historical dates

Context: Earnings announcements, especially when they exceed analyst expectations, can significantly impact stock prices in the short term.

Reference: Yahoo Finance earnings calendar - https://finance.yahoo.com/calendar/earnings?symbol=AMZN

*Additional*: Is there a correlation between the magnitude of the earnings surprise and the stock price reaction? Does the market react differently to earnings surprises during bull vs. bear markets?)


In [ ]:
# Load and clean earnings data
earnings = pd.read_csv('../../data/raw_data/ha1_Amazon.csv', sep=';').iloc[:-1, :]  # Exclude the last row which is a summary row
earnings['Earnings Date'] = pd.to_datetime(earnings['Earnings Date'].str.split(' at').str[0], errors='coerce')

# Clean numeric columns
numeric_cols = ['EPS Estimate', 'Reported EPS', 'Surprise (%)']
for col in numeric_cols:
    earnings[col] = pd.to_numeric(earnings[col].str.replace('[^-.0-9]', '', regex=True), errors='coerce')

# Download historical prices
amzn = yf.download('AMZN', start='1997-05-15')['Close'].reset_index()
amzn.columns = ['Date', 'Price']

# Calculate 2-day returns for ALL dates
amzn['2_day_pct_change'] =((amzn['Price'].shift(-2) / amzn['Price']) - 1)

positive_surprises = earnings[
    (earnings['Reported EPS'] > earnings['EPS Estimate']) | 
    (earnings['Surprise (%)'] > 0)
].copy()
#

dates_to_consider = positive_surprises['Earnings Date'].tolist()
# Filter AMZN data for the dates of interest
amzn_filtered = amzn[amzn['Date'].isin(dates_to_consider)].copy()

#( amzn_filtered['Price'].shift(-2)/amzn_filtered['Price'] -1).median()
amzn_filtered['2_day_pct_change'].median()*100

In [ ]:
# Merge earnings dates with price data
merged = pd.merge_asof(
    earnings.sort_values('Earnings Date'),
    amzn.sort_values('Date'),
    left_on='Earnings Date',
    right_on='Date',
    direction='forward'
)

# Filter positive earnings surprises
positive_surprises = merged[
    ((merged['Reported EPS'] > merged['EPS Estimate']) | (merged['Surprise (%)'] > 0)) &
    merged['2_day_pct_change'].notna()
].copy()

# Calculate median 2-day percentage changes
median_positive = positive_surprises['2_day_pct_change'].median()
median_all = amzn['2_day_pct_change'].median()

median_positive, median_all

In [ ]:
merged = pd.merge_asof(
    earnings.sort_values('Earnings Date'),
    amzn.sort_values('Date'),
    left_on='Earnings Date',
    right_on='Date',
    direction='forward',
    tolerance=pd.Timedelta('2D')
)
positive_surprises = merged[
    ((merged['Reported EPS'] > merged['EPS Estimate']) | 
    (merged['Surprise (%)'] > 0)) &
    merged['2_day_pct_change'].notna()
]

# Calculate comparison metrics
all_days_median = amzn['2_day_pct_change'].median()*100
positive_surprises_median = positive_surprises['2_day_pct_change'].median()*100

print(f"Median 2-day return after positive surprises: {positive_surprises_median:.2%}")
print(f"Median 2-day return for all trading days: {all_days_median:.2%}")

# Optional: Display key examples
print("\nNotable examples:")
print(positive_surprises[['Earnings Date', 'Surprise (%)', '2_day_pct_change']].head(10))



### Question 5.  [Exploratory, optional] Brainstorm potential idea for your capstone project

**Free text answer**

Describe the capstone project you would like to pursue, considering your aspirations, ML model predictions, and prior knowledge. Even if you are unsure at this stage, try to generate an idea you would like to explore-such as a specific asset class, country, industry vertical, or investment strategy. Be as specific as possible.

*Example: I want to build a short-term prediction model for the US/India/Brazil stock markets, focusing on the largest stocks over a 30-day investment horizon. I plan to use RSI and MACD technical indicators and news coverage data to generate predictions.*


**Answer:**
For my capstone project, I would like to develop a mid-term prediction model for equity markets in both Latin America (specifically Colombia, Brazil, Mexico, Argentina, Chile, Peru) and the United States. My focus will be on forecasting stock returns or index levels over a six-month investment horizon. I am interested in combining traditional technical indicators with macroeconomic variables, particularly each country’s interest rates and inflation rates, to improve predictive power.

My goal is to investigate how the interplay between technical signals and local macroeconomic conditions affects market performance, and whether machine learning models can capture these relationships for more robust investment strategies. As a physicist with no formal background in economics, I am motivated to bridge my quantitative skills with financial knowledge, and I look forward to learning more about macroeconomic indicators and their impact on asset prices as I progress through the project.

Potential challenges include sourcing reliable macroeconomic and market data for Latin American countries and interpreting the economic context behind the numbers. However, I believe this interdisciplinary approach could yield valuable insights for investors interested in emerging markets and global diversification.


### Question 6. [Exploratory, optional] Investigate new metrics

**Free text answer**

Using the data sources we have covered (or any others you find relevant), download and explore a few additional metrics or time series that could be valuable for your project. Briefly explain why you think each metric is useful. This does not need to be a comprehensive list-focus on demonstrating your ability to generate data requests based on your project description, identify and locate the necessary data, and explain how you would retrieve it using Python.